In [16]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from tensorflow.python.client import device_lib
import datetime

In [17]:
# Enable GPUs if present
print(device_lib.list_local_devices())
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5521946454118780974
xla_global_id: -1
]


In [18]:
# Define directories
data_folder = os.path.join("..", "data")
model_folder = os.path.join("..", "models")
tune_folder = os.path.join("..", "tuning")

# Seed
seed_value = 42

# Step 1: Data Processing

In [19]:
image_shape = (500, 500, 3)  # Create tuple for image size

# Establish paths
test_path = os.path.join(data_folder, "test")
train_path = os.path.join(data_folder, "train")
valid_path = os.path.join(data_folder, "valid")

# Load the images sets
test_images = keras.utils.image_dataset_from_directory(
    directory=test_path,
    labels="inferred",
    image_size=(image_shape[0], image_shape[1]),
)
train_images = keras.utils.image_dataset_from_directory(
    directory=train_path,
    labels="inferred",
    image_size=(image_shape[0], image_shape[1]),
)
valid_images = keras.utils.image_dataset_from_directory(
    directory=valid_path,
    labels="inferred",
    image_size=(image_shape[0], image_shape[1]),
)

Found 539 files belonging to 3 classes.
Found 1942 files belonging to 3 classes.
Found 431 files belonging to 3 classes.


# Step 2: Neural Network Architecture Design

In [20]:
def build_model(hp):
    # Initialize the model
    model = models.Sequential(
        [
            layers.Input(shape=image_shape),
            layers.RandomFlip("horizontal_and_vertical"),
            layers.RandomRotation(1),
            layers.RandomBrightness(0.2),
            layers.RandomContrast(0.2),
            layers.RandomCrop(400, 400),
            layers.RandomTranslation(0.2, 0.2),
            layers.RandomZoom(0.2),
        ]
    )

    # Tune the number of Conv2D layers (e.g., between 1 and 5 layers)
    for i in range(hp.Int("num_conv_layers", min_value=1, max_value=5)):
        model.add(
            layers.Conv2D(
                filters=hp.Int(
                    f"conv_{i+1}_filters", min_value=8, max_value=128, step=8
                ),
                kernel_size=hp.Choice(f"conv_{i+1}_kernel", values=[3, 5]),
                activation="relu",
            )
        )
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(
            layers.Dropout(
                rate=hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1)
            )
        )

    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())

    # Tune the number of Dense layers (e.g., between 1 and 5 layers)
    for j in range(hp.Int("num_dense_layers", min_value=1, max_value=5)):
        model.add(
            layers.Dense(
                units=hp.Int(
                    f"dense_{j+1}_units", min_value=10, max_value=100, step=10
                ),
                activation=hp.Choice("dense_activation", values=["relu", "selu"]),
            )
        )
        model.add(
            layers.Dropout(
                rate=hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1)
            )
        )

    model.add(layers.Dense(3, activation="softmax"))  # Output layer

    # Compile the model with a tunable learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Step 3: Hyperparameter Analysis

In [ ]:
# Create a tuner
# tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=50)
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=20,
    directory=tune_folder,
    project_name="bayesian_opt"
)

# Tune the model
tuner.search(
    train_images,
    epochs=10,
    validation_data=valid_images,
    batch_size=32
)

# Select the best model
model = tuner.get_best_models()[0]

# Display the model's architecture
model.summary()

In [ ]:
# Save the model
model_path = os.path.join(
    model_folder, f"model_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.keras"
)
model.save(model_path)

# Model Evaluation

In [26]:
# Load the model
model = keras.models.load_model(model_path)

# Evaluate on test data
test_loss, test_acc = model.evaluate(
    x=test_images
)

17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 205ms/step - accuracy: 0.5708 - loss: 1.1042
